In [ ]:
from modules.hand_visualizations import *
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

DATASET_PATH = "../2/rps-cv-images"

# --- REPRESENTATIONS TO TEST ---
REPRESENTATIONS = ['raw_rgb', 'landmarks', 'edges', 'seg_mask']


### ML Classificaiton

In [ ]:
if True:
    for rep in REPRESENTATIONS:
        print(f"\n=== Representation: {rep} ===")
        
        processor = HandVisualRepresentations()
        X, y = [], []
        for img, lab in zip(image_paths, labels_encoded):
            representations, _ = processor.process_rps_image(img)
            feat = representations.get(rep)
            if feat is not None:
                if rep == 'landmarks':
                    X.append(feat.flatten())
                elif rep == 'raw_rgb':
                    X.append(feat.flatten())
                else:
                    X.append(feat.flatten())
                y.append(lab)
        X = np.array(X)
        y = np.array(y)
        
        
        if len(X) > 10:
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
            scaler = StandardScaler()
            if rep != 'raw_rgb':  # Don't scale images
                X_train = scaler.fit_transform(X_train)
                X_test = scaler.transform(X_test)
            accs = {}
            for name, model in ML_MODELS.items():
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                acc = accuracy_score(y_test, y_pred)
                accs[name] = acc
                print(f"ML {name} accuracy: {acc:.3f}")
            results_ml[rep] = accs
        results_dl[rep] = {'train_loss': train_losses, 'test_acc': test_accs}


In [ ]:
# Plots
plt.figure(figsize=(10,6))
model_names = list(ML_MODELS.keys())
for rep in REPRESENTATIONS:
    if rep in results_ml:
        accs = [results_ml[rep][name] for name in model_names]
        plt.plot(model_names, accs, label=f"{rep}", marker='o')
plt.title("ML Model Accuracies per Representation")
plt.ylabel("Accuracy")
plt.xlabel("Model")
plt.legend()
plt.show()

In [ ]:
# accuracies as grouped barplot
plt.figure(figsize=(10,6))
model_names = list(ML_MODELS.keys())
bar_width = 0.18
x = np.arange(len(model_names))
for i, rep in enumerate(REPRESENTATIONS):
    if rep in results_ml:
        accs = [results_ml[rep][name] for name in model_names]
        colors = sns.color_palette("Set2", len(REPRESENTATIONS))
        plt.bar(x + i*bar_width, accs, width=bar_width, label=f"{rep}", color=colors[i])
        for j, v in enumerate(accs):
            plt.text(x[j] + i*bar_width, v + 0.01, f"{v:.2f}", ha='center', va='bottom', fontsize=9)
plt.xticks(x + bar_width*1.5, model_names)
plt.title("ML Model Accuracies per Representation")
plt.ylabel("Accuracy")
plt.xlabel("Model")
plt.legend(loc="lower right")
plt.show()

### DL Classification

In [ ]:

# --- DATASET CLASS ---
class HandDataset2(Dataset):
    def __init__(self, image_paths, labels, representation, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.representation = representation
        self.transform = transform
        self.processor = HandVisualRepresentations()
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        
        try:
            representations, _ = self.processor.process_rps_image(img_path)
            rep = representations.get(self.representation)
        except Exception as e:
            print(f"Warning: {e}")
            rep = None
            
        # Handle missing representations
        if rep is None:
            if self.representation == 'landmarks':
                rep = np.zeros(42)
            elif self.representation == 'raw_rgb':
                rep = np.zeros((224, 224, 3))
            else:
                rep = np.zeros((224, 224))
        
        # Convert to tensor
        if self.representation == 'landmarks':
            return torch.tensor(rep, dtype=torch.float32), torch.tensor(label, dtype=torch.long)
        elif self.representation == 'raw_rgb':
            if self.transform:
                rep = self.transform(rep)
            else:
                rep = torch.tensor(rep.transpose(2, 0, 1), dtype=torch.float32) / 255.
            return rep, torch.tensor(label, dtype=torch.long)
        else:
            rep = torch.tensor(rep, dtype=torch.float32)
            if rep.ndim == 2:
                rep = rep.unsqueeze(0)
            return rep, torch.tensor(label, dtype=torch.long)


# --- DATASET CLASS ---
class HandDataset(Dataset):
    def __init__(self, image_paths, labels, representation, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.representation = representation
        self.transform = transform
        self.processor = HandVisualRepresentations()
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        
        try:
            representations, _ = self.processor.process_rps_image(img_path)
            rep = representations.get(self.representation)
        except Exception as e:
            print(f"Warning: {e}")
            rep = None
            
        # Handle missing representations
        if rep is None:
            if self.representation == 'landmarks':
                rep = np.zeros(42)  # 21 landmarks × 2 coordinates
            elif self.representation == 'raw_rgb':
                rep = np.zeros((224, 224, 3))
            else:
                rep = np.zeros((224, 224))
        
        # Convert to tensor
        if self.representation == 'landmarks':
            # Ensure landmarks are properly shaped (21, 2) and flatten to (42,)
            if rep is not None:
                rep = np.array(rep)
                if rep.shape == (21, 3):  # If it has x, y, z coordinates
                    rep = rep[:, :2]  # Take only x, y coordinates
                elif rep.shape == (21, 2):  # Already correct shape
                    pass
                else:
                    print(f"Warning: Unexpected landmark shape {rep.shape}, using zeros")
                    rep = np.zeros((21, 2))
                
                # Flatten to 42 values
                rep = rep.flatten()
                
                # Ensure exactly 42 values
                if len(rep) != 42:
                    print(f"Warning: Landmark length {len(rep)}, padding/truncating to 42")
                    if len(rep) < 42:
                        rep = np.pad(rep, (0, 42 - len(rep)), 'constant', constant_values=0)
                    else:
                        rep = rep[:42]
            
            return torch.tensor(rep, dtype=torch.float32), torch.tensor(label, dtype=torch.long)
            
        elif self.representation == 'raw_rgb':
            if self.transform:
                rep = self.transform(rep)
            else:
                rep = torch.tensor(rep.transpose(2, 0, 1), dtype=torch.float32) / 255.
            return rep, torch.tensor(label, dtype=torch.long)
        else:
            rep = torch.tensor(rep, dtype=torch.float32)
            if rep.ndim == 2:
                rep = rep.unsqueeze(0)
            return rep, torch.tensor(label, dtype=torch.long)
        
        
# --- DL ARCHITECTURES ---

# 1. Simple MLP for landmarks
class LandmarkMLP(nn.Module):
    def __init__(self, input_size=42, num_classes=3):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        return self.fc3(x)

# 2. Deep MLP for landmarks
class DeepLandmarkMLP(nn.Module):
    def __init__(self, input_size=42, num_classes=3):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, num_classes)
        self.dropout = nn.Dropout(0.3)
        self.bn1 = nn.BatchNorm1d(256)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(64)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = F.relu(self.fc4(x))
        return self.fc5(x)

class SimpleCNN(nn.Module):
    def __init__(self, in_channels=3, num_classes=3):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((7, 7))  # Fixed output size
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # Now always 64*7*7 = 3136
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.adaptive_pool(x)  # Ensures consistent size
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)


class DeepCNN(nn.Module):
    def __init__(self, in_channels=3, num_classes=3):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))  # Fixed output size
        self.fc1 = nn.Linear(256 * 4 * 4, 512)  # Now always 256*4*4 = 4096
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(0.3)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
    
    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.adaptive_pool(x)  # Ensures consistent size
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        return self.fc3(x)
    

class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNetCNN(nn.Module):
    def __init__(self, in_channels=3, num_classes=3):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, 64, 7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(3, stride=2, padding=1)
        
        self.layer1 = ResNetBlock(64, 64)
        self.layer2 = ResNetBlock(64, 128, stride=2)
        self.layer3 = ResNetBlock(128, 256, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256, num_classes)
    
    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)


def train_model(model, train_loader, test_loader, epochs=5, device='cpu'):
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
    
    train_losses = []
    test_accs = []
    
    for epoch in range(epochs):
        # Training
        model.train()
        running_loss = 0.0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        train_losses.append(running_loss / len(train_loader))
        
        # Testing
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                pred = output.argmax(dim=1)
                correct += pred.eq(target).sum().item()
                total += target.size(0)
        
        accuracy = correct / total
        test_accs.append(accuracy)
        scheduler.step()
        
        print(f"Epoch {epoch+1}/{epochs}: Loss={train_losses[-1]:.4f}, Acc={accuracy:.3f}")
    
    return train_losses, test_accs


def run_dl_experiments():
    # Load dataset
    image_paths, labels, _ = explore_rps_dataset(DATASET_PATH)
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(labels)
    
    # Split dataset
    X_train_paths, X_test_paths, y_train, y_test = train_test_split(
        image_paths, labels_encoded, test_size=0.2, stratify=labels_encoded, random_state=42
    )
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    results = {}
    
    for rep in REPRESENTATIONS:
        print(f"\n{'='*50}")
        print(f"REPRESENTATION: {rep}")
        print(f"{'='*50}")
        
        # Define architectures for this representation
        if rep == 'landmarks':
            architectures = {
                'Simple MLP': LandmarkMLP(input_size=42, num_classes=3),
                'Deep MLP': DeepLandmarkMLP(input_size=42, num_classes=3)
            }
            transform = None
            batch_size = 32
        else:
            in_channels = 3 if rep == 'raw_rgb' else 1
            architectures = {
                'Simple CNN': SimpleCNN(in_channels=in_channels, num_classes=3),
                'Deep CNN': DeepCNN(in_channels=in_channels, num_classes=3),
                'ResNet CNN': ResNetCNN(in_channels=in_channels, num_classes=3)
            }
            transform = transforms.Compose([
                transforms.ToTensor() if rep == 'raw_rgb' else transforms.Lambda(lambda x: x),
                transforms.Resize((224, 224)),
                transforms.Normalize(mean=[0.5]*3, std=[0.5]*3) if rep == 'raw_rgb' else transforms.Normalize(mean=[0.5], std=[0.5])
            ])
            batch_size = 16
        
        # Create datasets
        train_dataset = HandDataset(X_train_paths, y_train, rep, transform)
        test_dataset = HandDataset(X_test_paths, y_test, rep, transform)
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        
        results[rep] = {}
        
        for arch_name, model in architectures.items():
            print(f"\nTraining {arch_name}...")
            train_losses, test_accs = train_model(model, train_loader, test_loader, epochs=5, device=device)
            results[rep][arch_name] = {
                'train_losses': train_losses,
                'test_accs': test_accs,
                'final_acc': test_accs[-1]
            }
    
    return results

def plot_results(results):
    # Plot 1: Final accuracies comparison
    num_reps = len(REPRESENTATIONS)
    fig_rows = 2
    fig_cols = max(2, (num_reps + 1) // 2 + 1)  # Adjust columns based on representations
    
    plt.figure(figsize=(15, 8))
    
    # Prepare data for plotting
    all_results = []
    for rep in results:
        for arch in results[rep]:
            all_results.append({
                'Representation': rep,
                'Architecture': arch,
                'Accuracy': results[rep][arch]['final_acc']
            })
    
    df = pd.DataFrame(all_results)
    
    # Create grouped bar plot
    plt.subplot(fig_rows, fig_cols, 1)
    sns.barplot(data=df, x='Architecture', y='Accuracy', hue='Representation')
    plt.title('Final Test Accuracy by Architecture and Representation')
    plt.xticks(rotation=45)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Plot 2: Training curves for each representation
    colors = plt.cm.Set1(np.linspace(0, 1, 10))
    
    for i, rep in enumerate(REPRESENTATIONS):
        if rep in results:  # Only plot if we have results for this representation
            plt.subplot(fig_rows, fig_cols, i+2)
            color_idx = 0
            for arch in results[rep]:
                epochs = range(1, len(results[rep][arch]['test_accs']) + 1)
                plt.plot(epochs, results[rep][arch]['test_accs'], 
                        label=arch, color=colors[color_idx], marker='o')
                color_idx += 1
            plt.title(f'{rep} - Test Accuracy over Epochs')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.legend()
            plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary table
    print("\n" + "="*80)
    print("FINAL RESULTS SUMMARY")
    print("="*80)
    print(f"{'Representation':<15} {'Architecture':<15} {'Final Accuracy':<15}")
    print("-"*80)
    
    for rep in results:
        for arch in results[rep]:
            acc = results[rep][arch]['final_acc']
            print(f"{rep:<15} {arch:<15} {acc:.3f}")

In [3]:
# --- RUN EXPERIMENTS ---
if __name__ == "__main__":
    results = run_dl_experiments()
    plot_results(results)

Dataset path: /Users/christina/code/RockPaperScissors/2/rps-cv-images
Dataset exists: True
Total images found: 2188

Class distribution:
paper: 712 images
rock: 726 images
scissors: 750 images

Sample image paths:
paper: /Users/christina/code/RockPaperScissors/2/rps-cv-images/paper/W79peyAyfQqNP1vF.png
rock: /Users/christina/code/RockPaperScissors/2/rps-cv-images/rock/foxUXc8WPRDAd6LM.png
scissors: /Users/christina/code/RockPaperScissors/2/rps-cv-images/scissors/6TMYdUMhaEWHQOcc.png
Using device: cpu

REPRESENTATION: raw_rgb


I0000 00:00:1752743246.608841   39470 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
I0000 00:00:1752743246.612763   39470 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro
W0000 00:00:1752743246.616749   39758 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1752743246.618922   39768 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1752743246.621555   39752 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1752743246.623469   39768 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedba


Training Simple CNN...
Epoch 1/5: Loss=1.0002, Acc=0.852
Epoch 2/5: Loss=0.3875, Acc=0.884
Epoch 3/5: Loss=0.2619, Acc=0.929
Epoch 4/5: Loss=0.1850, Acc=0.941
Epoch 5/5: Loss=0.1633, Acc=0.938

Training Deep CNN...
Epoch 1/5: Loss=1.2677, Acc=0.918
Epoch 2/5: Loss=0.2089, Acc=0.943
Epoch 3/5: Loss=0.1560, Acc=0.957
Epoch 4/5: Loss=0.1466, Acc=0.945
Epoch 5/5: Loss=0.0938, Acc=0.984

Training ResNet CNN...
Epoch 1/5: Loss=0.3528, Acc=0.893
Epoch 2/5: Loss=0.1384, Acc=0.904
Epoch 3/5: Loss=0.0820, Acc=0.986
Epoch 4/5: Loss=0.0524, Acc=0.977
Epoch 5/5: Loss=0.0669, Acc=0.986

REPRESENTATION: landmarks

Training Simple MLP...


I0000 00:00:1752746885.658174   39470 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro
W0000 00:00:1752746885.665390   68106 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1752746885.665653   39470 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro
W0000 00:00:1752746885.669623   68106 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1752746885.670574   68120 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1752746885.674822   68125 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


RuntimeError: mat1 and mat2 shapes cannot be multiplied (672x2 and 42x128)

In [9]:
if True:
    # Load dataset
    image_paths, labels, _ = explore_rps_dataset(DATASET_PATH)
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(labels)
    
    # Split dataset
    X_train_paths, X_test_paths, y_train, y_test = train_test_split(
        image_paths, labels_encoded, test_size=0.2, stratify=labels_encoded, random_state=42
    )
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # results = {}
    
    for rep in REPRESENTATIONS[2:]:
        print(f"\n{'='*50}")
        print(f"REPRESENTATION: {rep}")
        print(f"{'='*50}")
        
        # Define architectures for this representation
        if rep == 'landmarks':
            architectures = {
                'Simple MLP': LandmarkMLP(input_size=42, num_classes=3),
                #'Deep MLP': DeepLandmarkMLP(input_size=42, num_classes=3)
            }
            transform = None
            batch_size = 32
        else:
            in_channels = 3 if rep == 'raw_rgb' else 1
            architectures = {
                'Simple CNN': SimpleCNN(in_channels=in_channels, num_classes=3),
                #'Deep CNN': DeepCNN(in_channels=in_channels, num_classes=3),
                'ResNet CNN': ResNetCNN(in_channels=in_channels, num_classes=3)
            }
            transform = transforms.Compose([
                transforms.ToTensor() if rep == 'raw_rgb' else transforms.Lambda(lambda x: x),
                transforms.Resize((224, 224)),
                transforms.Normalize(mean=[0.5]*3, std=[0.5]*3) if rep == 'raw_rgb' else transforms.Normalize(mean=[0.5], std=[0.5])
            ])
            batch_size = 16
        
        # Create datasets
        train_dataset = HandDataset(X_train_paths, y_train, rep, transform)
        test_dataset = HandDataset(X_test_paths, y_test, rep, transform)
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        
        results[rep] = {}
        
        for arch_name, model in architectures.items():
            print(f"\nTraining {arch_name}...")
            train_losses, test_accs = train_model(model, train_loader, test_loader, epochs=5, device=device)
            results[rep][arch_name] = {
                'train_losses': train_losses,
                'test_accs': test_accs,
                'final_acc': test_accs[-1]
            }
    

Dataset path: /Users/christina/code/RockPaperScissors/2/rps-cv-images
Dataset exists: True
Total images found: 2188

Class distribution:
paper: 712 images
rock: 726 images
scissors: 750 images

Sample image paths:
paper: /Users/christina/code/RockPaperScissors/2/rps-cv-images/paper/W79peyAyfQqNP1vF.png
rock: /Users/christina/code/RockPaperScissors/2/rps-cv-images/rock/foxUXc8WPRDAd6LM.png
scissors: /Users/christina/code/RockPaperScissors/2/rps-cv-images/scissors/6TMYdUMhaEWHQOcc.png
Using device: cpu

REPRESENTATION: edges

Training Simple CNN...


I0000 00:00:1752750115.336934   39470 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro
I0000 00:00:1752750115.342863   39470 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro
W0000 00:00:1752750115.347086   97162 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1752750115.354933   97171 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1752750115.355558   97169 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1752750115.362508   97176 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Epoch 1/5: Loss=0.6884, Acc=0.833
Epoch 2/5: Loss=0.3965, Acc=0.863
Epoch 3/5: Loss=0.3021, Acc=0.870
Epoch 4/5: Loss=0.2517, Acc=0.895
Epoch 5/5: Loss=0.1986, Acc=0.904

Training ResNet CNN...
Epoch 1/5: Loss=0.3815, Acc=0.594
Epoch 2/5: Loss=0.2463, Acc=0.523
Epoch 3/5: Loss=0.2190, Acc=0.767
Epoch 4/5: Loss=0.1612, Acc=0.811
Epoch 5/5: Loss=0.1675, Acc=0.938

REPRESENTATION: seg_mask

Training Simple CNN...


I0000 00:00:1752752543.117589   39470 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro
I0000 00:00:1752752543.121260   39470 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro
W0000 00:00:1752752543.127305  127868 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1752752543.127988  127881 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1752752543.131767  127868 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1752752543.132273  127887 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Epoch 1/5: Loss=0.7386, Acc=0.815
Epoch 2/5: Loss=0.3794, Acc=0.920
Epoch 3/5: Loss=0.2266, Acc=0.959
Epoch 4/5: Loss=0.1617, Acc=0.952
Epoch 5/5: Loss=0.1571, Acc=0.968

Training ResNet CNN...
Epoch 1/5: Loss=0.3604, Acc=0.685
Epoch 2/5: Loss=0.1526, Acc=0.954
Epoch 3/5: Loss=0.0946, Acc=0.331
Epoch 4/5: Loss=0.1067, Acc=0.968
Epoch 5/5: Loss=0.0698, Acc=0.984
